In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/jageorge/Downloads/Run_2_PostedWorkOrders_11-Jan-21 23_31_06.csv')
df.drop(149, axis=0,inplace = True)
df

,Work Order Name,Post Submission Time,Post Start Time,Post Complete Time,Networks Or Edits,Bin,Operation
0,J0_02026,12/01/2021 06:04:59,11/01/2021 23:30:55,11/01/2021 23:31:07,2,1,EDIT
1,J2_00805,09/01/2021 00:58:40,11/01/2021 23:31:07,11/01/2021 23:31:18,1,1,CREATE
2,J0_08309,09/01/2021 00:58:50,11/01/2021 23:31:19,11/01/2021 23:31:27,3,1,EDIT
3,J0_08317,09/01/2021 00:58:51,11/01/2021 23:31:27,11/01/2021 23:31:35,1,1,EDIT
4,J0_07124,09/01/2021 00:58:54,11/01/2021 23:31:36,11/01/2021 23:31:45,1,1,EDIT
...,...,...,...,...,...,...,...
2890,J0_02884,12/01/2021 01:51:12,12/01/2021 13:37:11,12/01/2021 13:37:20,5,2,EDIT
2891,J0_06659,12/01/2021 01:51:15,12/01/2021 13:37:21,12/01/2021 13:37:34,2,1,EDIT
2892,J0_08058,12/01/2021 01:51:16,12/01/2021 13:37:34,12/01/2021 13:37:48,5,2,EDIT
2893,J0_00622,12/01/2021 01:51:24,12/01/2021 13:37:48,12/01/2021 13:37:58,20,5,EDIT


In [3]:
df.dtypes
df['PostSubTime'] = pd.to_datetime(df['Post Submission Time'] )
df['PostStartTime'] = pd.to_datetime(df['Post Start Time'] )
df['PostCompTime'] = pd.to_datetime(df['Post Complete Time'] )

In [6]:
df = df[['Bin','PostSubTime','PostStartTime','PostCompTime','Operation']]
df.head()

,Bin,PostSubTime,PostStartTime,PostCompTime,Operation
0,1,2021-12-01 06:04:59,2021-11-01 23:30:55,2021-11-01 23:31:07,EDIT
1,1,2021-09-01 00:58:40,2021-11-01 23:31:07,2021-11-01 23:31:18,CREATE
2,1,2021-09-01 00:58:50,2021-11-01 23:31:19,2021-11-01 23:31:27,EDIT
3,1,2021-09-01 00:58:51,2021-11-01 23:31:27,2021-11-01 23:31:35,EDIT
4,1,2021-09-01 00:58:54,2021-11-01 23:31:36,2021-11-01 23:31:45,EDIT


In [24]:

def change_hour(time):
    if time.hour >= start_hour:
        return (time.hour - start_hour + 1)
    else:
        return (time.hour - start_hour + 1+24)


In [27]:

df['Waittime'] = df['PostStartTime'].shift(periods=-1) - df['PostCompTime'] # need to shift to subtract last complete time
df['PostTime'] = df['PostCompTime'] - df['PostStartTime']
df['PostStartHour'] = df.PostStartTime.apply(change_hour)
df.head()

,Bin,PostSubTime,PostStartTime,PostCompTime,Operation,Waittime,PostTime,PostStartHour
0,1,2021-12-01 06:04:59,2021-11-01 23:30:55,2021-11-01 23:31:07,EDIT,00:00:00,00:00:12,1
1,1,2021-09-01 00:58:40,2021-11-01 23:31:07,2021-11-01 23:31:18,CREATE,00:00:01,00:00:11,1
2,1,2021-09-01 00:58:50,2021-11-01 23:31:19,2021-11-01 23:31:27,EDIT,00:00:00,00:00:08,1
3,1,2021-09-01 00:58:51,2021-11-01 23:31:27,2021-11-01 23:31:35,EDIT,00:00:01,00:00:08,1
4,1,2021-09-01 00:58:54,2021-11-01 23:31:36,2021-11-01 23:31:45,EDIT,00:00:00,00:00:09,1


In [28]:
df[df.PostStartHour == 1].PostTime.mean()

Timedelta('0 days 00:00:11.255033')

In [29]:
first = df.groupby(df.PostStartHour).agg({'Bin':['count'],
                                  'Waittime':lambda x: x.astype('timedelta64[s]').mean(),
                                 'PostTime':lambda x: x.astype('timedelta64[s]').mean()})
first.columns = ['Bin','AvgWaitTime','AvgPostTime']
first = first.reset_index()
first.to_csv('Hour_summary.csv')
first.head()

,PostStartHour,Bin,AvgWaitTime,AvgPostTime
0,1,149,16816.563758,11.255034
1,2,277,0.180505,12.830325
2,3,246,0.219512,14.414634
3,4,226,0.424779,15.504425
4,5,228,0.421053,15.381579


In [30]:
df2 = df.groupby('Operation').agg({'PostTime':lambda x: x.astype('timedelta64[s]').mean()})
df2.to_csv('create_edit.csv')
df2.head()

,PostTime
Operation,
CREATE,32.110619
EDIT,14.114251


In [32]:
lst = df.groupby(['Operation','Bin'])['PostSubTime'].count()
lst.name = 'Count'
lst = lst.reset_index()
lst.to_csv('bin_operation.csv')
lst

,Operation,Bin,Count
0,CREATE,1,153
1,CREATE,2,68
2,CREATE,3,70
3,CREATE,4,41
4,CREATE,5,33
5,CREATE,6,28
6,CREATE,7,10
7,CREATE,8,6
8,CREATE,9,21
9,CREATE,10,7


In [33]:
createtotal = df[df.Operation=='CREATE']['Bin'].count()
edittotal = df[df.Operation=='EDIT']['Bin'].count()
#print(createtotal,edittotal)

def funct(row):
    if row['Operation']=='CREATE':
        return row['Bin']/createtotal
    else:
        return row['Bin']/edittotal

In [34]:
lst['Percent'] = lst.apply(lambda row: funct(row), axis=1)
lst.to_csv('percent.csv')
lst.head()

,Operation,Bin,Count,Percent
0,CREATE,1,153,0.002212
1,CREATE,2,68,0.004425
2,CREATE,3,70,0.006637
3,CREATE,4,41,0.008850
4,CREATE,5,33,0.011062


In [35]:
newdf = df.groupby(['PostStartHour','Operation','Bin'])['PostSubTime'].count()
newdf.name = 'Count'
newdf = newdf.reset_index()
newdf.to_csv('hour_operation_bin.csv')
newdf.head()

,PostStartHour,Operation,Bin,Count
0,1,CREATE,1,8
1,1,CREATE,2,2
2,1,CREATE,3,2
3,1,CREATE,4,1
4,1,CREATE,5,1
